# 结合其他系数进行楼层修正
$$房屋价格 = 小区价格\times朝向\times成新\times结构\times面积\times楼层$$
$$楼层 = 房屋价格/(小区价格\times朝向\times成新\times结构\times面积)$$

或者先使用其他系数将房价还原，然后就可以得到楼层系数对房价的影响

$$未修正小区价格 = 房屋价格/(朝向\times 成新 \times 结构 \times 面积)$$
$$未修正小区价格 = 小区价格\times 楼层修正系数$$
$$k = 未修正小区价格/小区价格$$
$$x = 楼层|楼层位置$$
$$K = ax^2 + bx + c$$

In [1]:
import pandas as pd
from tqdm import tqdm
house = pd.read_excel("D:\SUZHOU_CHANGE_EQUATION\宿州市住宅类可比实例入库版20180529.xlsx")
chaoxiang = pd.read_excel("D:\SUZHOU_CHANGE_EQUATION\房屋修正系数.xls",sheet_name="朝向")
chaoxiang = chaoxiang[chaoxiang['DS'] == 'SZ']
chengxin = pd.read_excel("D:\SUZHOU_CHANGE_EQUATION\房屋修正系数.xls",sheet_name="成新")
chengxin = chengxin[chengxin['DS']=='SZ']
jiegou = pd.read_excel("D:\SUZHOU_CHANGE_EQUATION\房屋修正系数.xls",sheet_name="结构")
jiegou = jiegou[jiegou['DS']=='SZ']
mianji = pd.read_excel("D:\SUZHOU_CHANGE_EQUATION\房屋修正系数.xls",sheet_name="面积")
mianji = mianji[mianji['DS']=='SZ']
chengxin

,DS,FCLX,JZCX_MIN,JZCX_MAX,XZXS
690,SZ,独门院落,16.0,20,99.0
691,SZ,独门院落,21.0,25,97.0
692,SZ,独门院落,26.0,30,96.0
693,SZ,独门院落,31.0,100,96.0
694,SZ,独栋别墅,-0.1,5,104.0
695,SZ,独栋别墅,6.0,10,102.0
696,SZ,独栋别墅,11.0,15,100.0
697,SZ,独栋别墅,16.0,20,98.0
698,SZ,独栋别墅,21.0,25,96.0
699,SZ,独栋别墅,26.0,30,90.0


In [2]:
house

,城市,行政区域,类型,小区名称,地址,所在楼层,总楼层,面积,结构,建筑年代,朝向,装修,买卖时间,单价
0,宿州市,埇桥区,多层住宅,南翔宝贝城（开发区）3#-6#(6楼),淮海路与迎宾大道交叉口向南300米,6,6,86.00,混合结构,2016,两间朝南,毛坯,2017-07-11,2800.0
1,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,1,18,73.31,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0
2,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,2,18,63.35,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0
3,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,3,18,85.63,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0
4,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,4,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0
5,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,5,18,63.86,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0
6,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,6,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2720.0
7,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,7,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2740.0
8,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,8,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2760.0
9,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,9,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2780.0


### 朝向系数计算

In [3]:
chaoxiang['朝向'] = chaoxiang['CX']
chaoxiang['类型'] = chaoxiang['FCLX']
chaoxiang['朝向系数'] = chaoxiang['XZXS']
chaoxiang_ = chaoxiang[['朝向','类型','朝向系数']]
house = pd.merge(house,chaoxiang_,how='left')
house

,城市,行政区域,类型,小区名称,地址,所在楼层,总楼层,面积,结构,建筑年代,朝向,装修,买卖时间,单价,朝向系数
0,宿州市,埇桥区,多层住宅,南翔宝贝城（开发区）3#-6#(6楼),淮海路与迎宾大道交叉口向南300米,6,6,86.00,混合结构,2016,两间朝南,毛坯,2017-07-11,2800.0,100.0
1,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,1,18,73.31,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0
2,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,2,18,63.35,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0
3,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,3,18,85.63,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0
4,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,4,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0
5,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,5,18,63.86,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0
6,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,6,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2720.0,100.0
7,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,7,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2740.0,100.0
8,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,8,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2760.0,100.0
9,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,9,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2780.0,100.0


### 成新系数计算

In [4]:
chengxin['类型'] = chengxin['FCLX']
chengxinxishu = []
for i,row in tqdm(house.iterrows()):
    cx = chengxin[(chengxin['类型'] == row['类型'])&(chengxin['JZCX_MIN'] <= 2018 - row['建筑年代'])&(chengxin['JZCX_MAX'] >= 2018 - row['建筑年代'])]['XZXS'].values[0]
    chengxinxishu.append(cx)
house['成新系数'] = chengxinxishu
house

10712it [00:28, 377.01it/s]


,城市,行政区域,类型,小区名称,地址,所在楼层,总楼层,面积,结构,建筑年代,朝向,装修,买卖时间,单价,朝向系数,成新系数
0,宿州市,埇桥区,多层住宅,南翔宝贝城（开发区）3#-6#(6楼),淮海路与迎宾大道交叉口向南300米,6,6,86.00,混合结构,2016,两间朝南,毛坯,2017-07-11,2800.0,100.0,103.0
1,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,1,18,73.31,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0
2,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,2,18,63.35,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0
3,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,3,18,85.63,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0
4,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,4,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0
5,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,5,18,63.86,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0
6,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,6,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2720.0,100.0,103.0
7,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,7,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2740.0,100.0,103.0
8,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,8,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2760.0,100.0,103.0
9,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,9,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2780.0,100.0,103.0


### 结构系数修正

In [5]:
jiegou['结构'] = jiegou['JZJG']
jiegou['类型'] = jiegou['FCLX']
jiegou['结构系数'] = jiegou['XZXS']
jiegou_ = jiegou[['结构','类型','结构系数']]
house = pd.merge(house,jiegou_,how='left')
house

,城市,行政区域,类型,小区名称,地址,所在楼层,总楼层,面积,结构,建筑年代,朝向,装修,买卖时间,单价,朝向系数,成新系数,结构系数
0,宿州市,埇桥区,多层住宅,南翔宝贝城（开发区）3#-6#(6楼),淮海路与迎宾大道交叉口向南300米,6,6,86.00,混合结构,2016,两间朝南,毛坯,2017-07-11,2800.0,100.0,103.0,100
1,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,1,18,73.31,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102
2,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,2,18,63.35,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102
3,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,3,18,85.63,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102
4,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,4,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102
5,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,5,18,63.86,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102
6,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,6,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2720.0,100.0,103.0,102
7,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,7,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2740.0,100.0,103.0,102
8,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,8,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2760.0,100.0,103.0,102
9,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,9,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2780.0,100.0,103.0,102


### 面积系数修正

In [6]:
mianji

,DS,FCLX,JZMJ_MIN,JZMJ_MAX,XZXS
298,SZ,低层住宅,0,60,98.0
299,SZ,低层住宅,60,90,98.0
300,SZ,低层住宅,90,144,100.0
301,SZ,低层住宅,144,10000,95.0
302,SZ,多层住宅,0,60,98.0
303,SZ,多层住宅,60,90,98.0
304,SZ,多层住宅,90,144,100.0
305,SZ,多层住宅,144,10000,95.0
306,SZ,中高层住宅,0,60,98.0
307,SZ,中高层住宅,60,90,98.0


In [7]:
mianji['类型'] = mianji['FCLX']
mianjixishu = []
for i,row in tqdm(house.iterrows()):
    mj = mianji[(mianji['类型'] == row['类型'])&(mianji['JZMJ_MIN'] <= row['面积'])&(mianji['JZMJ_MAX'] >= row['面积'])]['XZXS'].values[0]
    mianjixishu.append(mj)
house['面积系数'] = mianjixishu
house['装修系数'] = 98
house

10712it [00:28, 375.75it/s]


,城市,行政区域,类型,小区名称,地址,所在楼层,总楼层,面积,结构,建筑年代,朝向,装修,买卖时间,单价,朝向系数,成新系数,结构系数,面积系数,装修系数
0,宿州市,埇桥区,多层住宅,南翔宝贝城（开发区）3#-6#(6楼),淮海路与迎宾大道交叉口向南300米,6,6,86.00,混合结构,2016,两间朝南,毛坯,2017-07-11,2800.0,100.0,103.0,100,98.0,98
1,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,1,18,73.31,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102,98.0,98
2,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,2,18,63.35,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102,98.0,98
3,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,3,18,85.63,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102,98.0,98
4,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,4,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102,98.0,98
5,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,5,18,63.86,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2700.0,100.0,103.0,102,98.0,98
6,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,6,18,86.37,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2720.0,100.0,103.0,102,98.0,98
7,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,7,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2740.0,100.0,103.0,102,98.0,98
8,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,8,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2760.0,100.0,103.0,102,98.0,98
9,宿州市,埇桥区,中高层住宅,百丽苑(开发区）1#-15#(18楼),外环二路南侧，大陈沟东侧,9,18,86.28,钢筋混凝土结构,2013,两间朝南,毛坯,2017-07-05,2780.0,100.0,103.0,102,98.0,98


In [8]:
house.to_excel("D:/SUZHOU_CHANGE_EQUATION/可比实例带系数.xlsx",index=False)